In [ ]:
import csv
from lxml import etree
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import nltk

header = ['Title', 'Text']
f= open("file.csv", "w+")
writer = csv.writer(f)
writer.writerow(header)
lista = []

totalCount = 0
for event, elem in etree.iterparse("file.xml"):
    text = ""
    title=""
    if('title' in str(elem.tag)):
        #title_list.append(elem.text)
        #titoliTxt.write(elem.text+"\n")
        if(len(str(elem.text)) > 0 and str(elem.text) != None):
            title = elem.text
            lista.append(title)
    if('text' in str(elem.tag)):
        #text_list.append(elem.text)
        if(len(str(elem.text)) > 0 and str(elem.text) != None):
            #paroleTxt.write(elem.text+"\n")
            text = elem.text
            stop_words = set(stopwords.words('english'))
            word_tokens = word_tokenize(elem.text)
            
            filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
            for w in word_tokens:
                if w not in stop_words:
                    filtered_sentence.append(w)
            print(str(filtered_sentence))
            #lista.append(str(filtered_sentence))
            '''lista.append(elem.text)
    if len(lista) ==2:
        writer.writerow([lista[0], lista[1]])'''
    lista=[]
    elem.clear()
    totalCount += 1
    if totalCount > 1 and (totalCount % 100000) == 0:
            print("{:,}".format(totalCount)+" rows written of 356,900,000")
    if totalCount > 35000:
        break
f.close()
print("-------------------------------------------DONE------------------------------------------------------")


# TODO: clear up wikipedia dump from markup code

In [ ]:
import pandas as pd

df = pd.read_csv("file.csv", low_memory=True)
print(df.head(5))

In [ ]:
import nltk
nltk.download('stopwords')
stops = set(stopwords.words('english'))
print(stops)